# 高斯雜訊產生器
此 Notebook 示範如何使用 Box–Muller 演算法產生含有高斯雜訊的影像，並視覺化結果。

## 匯入所需的函式庫
以下函式庫用於影像處理、視覺化以及建立互動式介面。

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import gradio as gr

## 定義高斯雜訊產生函式
此函式使用 Box–Muller 演算法產生含有高斯雜訊的影像，並回傳原始影像、含雜訊影像以及直方圖。

In [3]:
def generate_noisy_image(width: int, height: int, sigma: float):
    # 1. Create a grayscale image g(x,y) with all pixel values set to 100
    g = np.full((height, width), 100.0, dtype=np.float32)
    
    # 2. Generate Gaussian noise using the Box–Muller algorithm
    noise = np.zeros_like(g)
    for i in range(height):
        j = 0
        while j + 1 < width:
            r = np.random.rand()
            phi = np.random.rand()
            z1 = sigma * np.cos(2 * np.pi * phi) * np.sqrt(-2 * np.log(r))
            z2 = sigma * np.sin(2 * np.pi * phi) * np.sqrt(-2 * np.log(r))
            noise[i, j]   = z1
            noise[i, j+1] = z2
            j += 2
        if width % 2 == 1:
            r = np.random.rand()
            phi = np.random.rand()
            z = sigma * np.cos(2 * np.pi * phi) * np.sqrt(-2 * np.log(r))
            noise[i, -1] = z

    # 3. Add noise to the image and clip values to [0, 255]
    f = g + noise
    f = np.clip(f, 0, 255).astype(np.uint8)
    g_uint8 = np.clip(g, 0, 255).astype(np.uint8)
    
    # 4. Plot the histogram of the noisy image
    fig, ax = plt.subplots()
    ax.hist(f.flatten(), bins=256, range=(0,255))
    ax.set_title("Histogram of f(x,y)")
    ax.set_xlabel("Gray level")
    ax.set_ylabel("Frequency")
    plt.tight_layout()
    plt.close(fig)
    
    # Return the original image, noisy image, and histogram
    return Image.fromarray(g_uint8), Image.fromarray(f), fig

## 建立 Gradio 介面
Gradio 介面允許使用者互動式調整影像尺寸與雜訊強度 (σ)。

In [4]:
iface = gr.Interface(
    fn=generate_noisy_image,
    inputs=[
        gr.Slider(64, 512, value=256, step=1, label="Width"),
        gr.Slider(64, 512, value=256, step=1, label="Height"),
        gr.Slider(0.0, 50.0, value=5.0, step=0.1, label="Sigma (σ)")
    ],
    outputs=[
        gr.Image(type="pil", label="Input image g(x,y)"),
        gr.Image(type="pil", label="Noisy image f(x,y)"),
        gr.Plot(label="Histogram of f(x,y)")
    ],
    title="Gaussian Noise Generator",
    description="""
    本範例依照 Box–Muller 演算法為每對水平相鄰像素產生一對高斯雜訊，σ²=25 → σ=5  
    可自由調整影像尺寸與 σ，並即時顯示結果與直方圖。
    """
)

## 啟動介面
執行以下程式碼以啟動 Gradio 介面。

In [5]:
if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.

To create a public link, set `share=True` in `launch()`.
